## Creating TFRecord files for UNet-based segmentation

U-Net: Convolutional Networks for Biomedical Image Segmentation  
Olaf Ronneberger, Philipp Fischer and Thomas Brox  
http://arxiv.org/abs/1505.04597


### DATA ORGANISATION

The image list is used like this:

```python
images = ['phase','rfp']

set1/
  phase/
    
  gfp/
    0001_gfp.tif
    0002_gfp.tif
    ...
  rfp/
    0001_rfp.tif
    0002_rfp.tif
  ...
  label/
    0001_mask.tif
    0002_mask.tif
    ...
  weights/
    0001_weights.tif  # NOTE(arl): these are calculated
    0002_weights.tif
    ...
set2/
```

### STEPS
1. Calculate the weightmaps for the images 
2. Create the TFRecord file


### Set up the parameters

In [1]:
# root_dir = "/mnt/lowe-sn00/TrainingData/competition_fCNN"
# import_dirs = ["set1","set2"]

# root_dir = "/media/arl/DataII/Data/competition/training/UNet_Training_Jasmine"
# import_dirs = ["17_02_28", "17_01_24", "17_02_28", "17_02_28b", "17_02_28c", "17_03_31", "19_06_04",  "19_06_10", "19_06_10b"]
# import_dirs = ["19_06_10", "19_06_10b"]

#root_dir = "/home/nathan/analysis/training_data/jasmine"
root_dir = "/home/nathan/analysis/SHARC/training_data/images"
# import_dirs = ["set1", "set2", "set3"]
import_dirs = ["set{}".format(i+1) for i in range(16)]

w0 = 16.    # weight amplitude
sigma = 3.  # weight spatial stdev


#channels = ['gfp','rfp']
channels = ["phase"]

# training data filename
#frecord_fn = "train_competition_Ras_UNet_v2.tfrecord"
tfrecord_fn = "train_SHARC_w16s3.tfrecord"
 

Import the libraries

In [2]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import tifffile as tiff 
import weightmap

tifffile.py:2170: UserWarning: failed to import _tifffile.decodepackbits
  warnings.warn("failed to import %s" % module_function)
tifffile.py:2170: UserWarning: failed to import _tifffile.decodelzw
  warnings.warn("failed to import %s" % module_function)
tifffile.py:2170: UserWarning: failed to import _tifffile.unpackints
  warnings.warn("failed to import %s" % module_function)


Helper functions to find files and folders

In [4]:
def find_files(pth, stem='', ext='.tif'):
    """ Return a list of files with a certain stem and extension """
    if not os.path.exists(pth): return []
    files = os.listdir(pth)
    return [f for f in files if f.startswith(stem) and f.endswith(ext)]

def find_folders(pth):
    """ Return a list of folders in a defined path """
    folders = os.listdir(pth)
    return [f for f in folders if os.path.isdir(os.path.join(pth,f))]

A function to write out the TFRecord file used by the server to train the models

In [5]:
def write_tfrecord(pth,
                   filename,
                   images=['phase'],
                   weights_dir_base='weights'):
    
    try:
        import tensorflow as tf
    except ImportError:
        raise ImportError("Tensorflow is not installed.")
    
    # _int64 is used for numeric values
    def _int64_feature(value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

    # _bytes is used for string/char values
    def _bytes_feature(value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

    # _floats is used for float values
    def _float_feature(value):
        return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    
    channels = ('brightfield','phase','gfp','rfp','irfp')

    if any([i for i in images if i not in channels]):
        raise ValueError('Image type is not recognised')

    # set up the writer
    writer = tf.python_io.TFRecordWriter(os.path.join(pth, filename))

    # label
    pattern = '([a-zA-Z0-9()]+)_([a-zA-Z0-9()]+_)*'
    file_root = lambda x: re.match(pattern, x).group(0)

    folders = find_folders(pth)
    if 'HDF' in folders:
        folders.remove('HDF')   # remove any label folders from the list

    # a counter for the number of files
    n_files = 0

    for folder in folders:

        # root_folders
        l_root = os.path.join(pth, folder, 'label/')
        w_root = os.path.join(pth, folder, weights_dir_base)

        # get the training label files
        label_files = find_files(l_root)

        # now get the matching image files and trim to match labels
        # image files is a list of e.g. phase/0001_phase.tif
        image_files = [[i+'/'+file_root(l)+i+'.tif' for i in images] for l in label_files]
        weight_files = [file_root(l)+'weights.tif' for l in label_files]
        assert(len(label_files) == len(weight_files) == len(image_files))

        # zip them so we can iterate over pairs
        files = zip(*(image_files, label_files, weight_files))

        for im, l, w in files:

            i_data = None
            for i in im:
                if i_data is None:
                    i_data = tiff.imread(os.path.join(pth, folder, i))
                else:
                    i_data = np.stack((i_data, tiff.imread(os.path.join(pth, folder, i))), axis=-1)

            # squeeze to remove singleton dimensions
            #i_data = np.squeeze(i_data).astype('float32')
            i_data = np.squeeze(i_data).astype('uint8')

            # now read the labels and weights
            im_labels = weightmap.ImageLabels(os.path.join(l_root, l))
            l_data = im_labels.labels()
            outputs = im_labels.outputs

            # get the weights
            w_data = tiff.imread(os.path.join(w_root, w)).astype('float32')
            
            print folder, im, l, w

            if i_data.ndim == 2:
                depth = 1
            elif i_data.ndim == 3:
                depth = i_data.shape[-1]
            else:
                print i_data.shape
                raise Exception("Image data is not 2D or 3D.")

            # check that all of the images are the same size
            assert(i_data.shape[0:2] == l_data.shape[0:2] == w_data.shape[0:2])

           
            print 'Input: {0:s}'.format(str(i_data.shape)),
            print '--> Output: {0:s}'.format(str((l_data.shape[0], l_data.shape[1], outputs)))

            # set up TF feature dict
            feature = {'train/image/image': _bytes_feature(i_data.tostring()),
                       'train/image/width': _int64_feature(i_data.shape[1]),
                       'train/image/height': _int64_feature(i_data.shape[0]),
                       'train/image/depth': _int64_feature(depth),
                       'train/label/image': _bytes_feature(l_data.tostring()),
                       'train/label/width': _int64_feature(l_data.shape[1]),
                       'train/label/height': _int64_feature(l_data.shape[0]),
                       'train/label/depth': _int64_feature(outputs),
                       'train/weight/image': _bytes_feature(w_data.tostring()),
                       'train/weight/width': _int64_feature(w_data.shape[1]),
                       'train/weight/height': _int64_feature(w_data.shape[0]),
                       'train/weight/depth': _int64_feature(1)}

            features = tf.train.Features(feature=feature)
            example = tf.train.Example(features=features)

            # write out the serialized features
            writer.write(example.SerializeToString())

            n_files+=1

    # close up shop
    writer.close()

    print 'Written out {0:d} training images of size {1:s} to {2:s}'.format(n_files, str(i_data.shape), filename)

## DO THE PROCESSING
### 1. Create the weightmaps

In [6]:
ex = weightmap.create_weightmaps(root_dir, import_dirs, w0=w0, sigma=sigma, name_weights_folder=True)

Calculating weights for 0100_mask.tif in folder 'set1'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0000_mask.tif in folder 'set1'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0225_mask.tif in folder 'set1'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0031_mask.tif in folder 'set2'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0000_mask.tif in folder 'set3'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0321_mask.tif in folder 'set4'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0000_mask.tif in folder 'set4'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0441_mask.tif in folder 'set4'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0205_mask.tif in folder 'set4'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0265

In [7]:
# plt.figure(figsize=(16,10))
# plt.imshow(ex)
# plt.colorbar()
# plt.title("Example weightmap")
# plt.show()

### 2. Write out the TFRecord file

In [8]:
# set the correct weights folder
weights_dir_base = "weights_w0-{0:2.2f}_sigma-{1:2.2f}".format(w0, sigma)

In [9]:
write_tfrecord(root_dir, tfrecord_fn, images=channels, weights_dir_base=weights_dir_base)

(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set15 ['phase/0045_phase.tif'] 0045_mask.tif 0045_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set15 ['phase/0845_phase.tif'] 0845_mask.tif 0845_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set15 ['phase/1091_phase.tif'] 1091_mask.tif 1091_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set15 ['phase/0834_phase.tif'] 0834_mask.tif 0834_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set15 ['phase/0802_phase.tif'] 0802_mask.tif 0802_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set15 ['phase/0303_phase.tif'] 0303_mask.tif 0303_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressi

In [10]:
w0_array = [3., 7., 11., 16.]
sigma_array = [3., 7., 11., 16.]

for w0 in w0_array:
    for sigma in sigma_array:
        weightmap.create_weightmaps(root_dir, import_dirs, w0=w0, sigma=sigma, name_weights_folder=True)
        weights_dir_base = "weights_w0-{0:2.2f}_sigma-{1:2.2f}".format(w0, sigma)
        tfrecord_fn = "train_competition_Ras_UNet_v3_w0-{0:2.2f}_sigma-{1:2.2f}.tfrecord".format(w0, sigma)
        write_tfrecord(root_dir, tfrecord_fn, images=channels, weights_dir_base=weights_dir_base)
        
        



Calculating weights for 0100_mask.tif in folder 'set1'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0000_mask.tif in folder 'set1'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0225_mask.tif in folder 'set1'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0031_mask.tif in folder 'set2'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0000_mask.tif in folder 'set3'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0321_mask.tif in folder 'set4'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0000_mask.tif in folder 'set4'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0441_mask.tif in folder 'set4'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0205_mask.tif in folder 'set4'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0265

(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set16 ['phase/0000_phase.tif'] 0000_mask.tif 0000_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set16 ['phase/0001_phase.tif'] 0001_mask.tif 0001_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set11 ['phase/0625_phase.tif'] 0625_mask.tif 0625_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set11 ['phase/0000_phase.tif'] 0000_mask.tif 0000_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set10 ['phase/0502_phase.tif'] 0502_mask.tif 0502_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set10 ['phase/1092_phase.tif'] 1092_mask.tif 1092_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressi

Calculating weights for 0009_mask.tif in folder 'set5'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0000_mask.tif in folder 'set5'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0002_mask.tif in folder 'set5'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Creating output folder /home/nathan/analysis/SHARC/training_data/images/set6/weights_w0-3.00_sigma-7.00...
Calculating weights for 0009_mask.tif in folder 'set6'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0004_mask.tif in folder 'set6'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Creating output folder /home/nathan/analysis/SHARC/training_data/images/set7/weights_w0-3.00_sigma-7.00...
Calculating weights for 0229_mask.tif in folder 'set7'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Creating output folder /home/nathan/analysis/SHARC/training_data/images/set8/weights_w0-3.00_sigma-7.00...
Calculating weights for 0000_ma

(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set4 ['phase/0441_phase.tif'] 0441_mask.tif 0441_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set4 ['phase/0205_phase.tif'] 0205_mask.tif 0205_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set4 ['phase/0265_phase.tif'] 0265_mask.tif 0265_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set7 ['phase/0229_phase.tif'] 0229_mask.tif 0229_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set13 ['phase/0659(0303b)_phase.tif'] 0659(0303b)_mask.tif 0659(0303b)_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set13 ['phase/0409(0053b)_phase.tif'] 0409(0053b)_mask.tif 0409(0053b)_weights.tif
Input: (1024, 1024) --> Output: 

Creating output folder /home/nathan/analysis/SHARC/training_data/images/set10/weights_w0-3.00_sigma-11.00...
Calculating weights for 0502_mask.tif in folder 'set10'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 1092_mask.tif in folder 'set10'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0000_mask.tif in folder 'set10'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Creating output folder /home/nathan/analysis/SHARC/training_data/images/set11/weights_w0-3.00_sigma-11.00...
Calculating weights for 0625_mask.tif in folder 'set11'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0000_mask.tif in folder 'set11'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Creating output folder /home/nathan/analysis/SHARC/training_data/images/set12/weights_w0-3.00_sigma-11.00...
Calculating weights for 1090_mask.tif in folder 'set12'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights

...

Calculating weights for 0144_mask.tif in folder 'set14'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Creating output folder /home/nathan/analysis/SHARC/training_data/images/set15/weights_w0-16.00_sigma-7.00...
Calculating weights for 0045_mask.tif in folder 'set15'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0845_mask.tif in folder 'set15'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 1091_mask.tif in folder 'set15'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0834_mask.tif in folder 'set15'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0802_mask.tif in folder 'set15'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0303_mask.tif in folder 'set15'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0000_mask.tif in folder 'set15'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights

Compressing labels from [0 1] to [0, 1]
set9 ['phase/0441_phase.tif'] 0441_mask.tif 0441_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set9 ['phase/0020_phase.tif'] 0020_mask.tif 0020_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set5 ['phase/0005_phase.tif'] 0005_mask.tif 0005_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set5 ['phase/0009_phase.tif'] 0009_mask.tif 0009_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set5 ['phase/0000_phase.tif'] 0000_mask.tif 0000_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set5 ['phase/0002_phase.tif'] 0002_mask.tif 0002_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1

Calculating weights for 0997_mask.tif in folder 'set15'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 1000_mask.tif in folder 'set15'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0521_mask.tif in folder 'set15'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Creating output folder /home/nathan/analysis/SHARC/training_data/images/set16/weights_w0-16.00_sigma-11.00...
Calculating weights for 0000_mask.tif in folder 'set16'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0001_mask.tif in folder 'set16'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set15 ['phase/0045_phase.tif'] 0045_mask.tif 0045_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set15 ['phase/0845_phase.tif'] 0845_mask.tif 0845_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compre

Calculating weights for 0000_mask.tif in folder 'set1'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0225_mask.tif in folder 'set1'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Creating output folder /home/nathan/analysis/SHARC/training_data/images/set2/weights_w0-16.00_sigma-16.00...
Calculating weights for 0031_mask.tif in folder 'set2'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Creating output folder /home/nathan/analysis/SHARC/training_data/images/set3/weights_w0-16.00_sigma-16.00...
Calculating weights for 0000_mask.tif in folder 'set3'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Creating output folder /home/nathan/analysis/SHARC/training_data/images/set4/weights_w0-16.00_sigma-16.00...
Calculating weights for 0321_mask.tif in folder 'set4'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0000_mask.tif in folder 'set4'
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
Calculating weights for 0

set16 ['phase/0000_phase.tif'] 0000_mask.tif 0000_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set16 ['phase/0001_phase.tif'] 0001_mask.tif 0001_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set11 ['phase/0625_phase.tif'] 0625_mask.tif 0625_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set11 ['phase/0000_phase.tif'] 0000_mask.tif 0000_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set10 ['phase/0502_phase.tif'] 0502_mask.tif 0502_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set10 ['phase/1092_phase.tif'] 1092_mask.tif 1092_weights.tif
Input: (1024, 1024) --> Output: (1024, 1024, 2)
(1024, 1024)
Compressing labels from [0 1] to [0, 1]
set10 ['phase/0000_pha